In [ ]:
import os
import numpy as np
from scapy.all import rdpcap, raw, hexdump
import glob
import scipy.io
import random
import statistics

***adjust variables according to your case

att_cnt
- meta features to extract

uptopacket
- max number of packets in a flow to be processed (depend on resource constraint)

In [ ]:
att_cnt = 7
"""
0: sport
1: dport
2: payload mean (.payload)
3: payload SD
4: packet length mean
5: packet length SD
6: flow duration (.time)
"""

uptopacket=100

***the below function takes in a .pcap file and return
(1) timestamps, (2) flow bytes after prepocessing, and (3) meta fetures of a flow

In [ ]:
def read_raw(pcapFilename):
    
    global_att = np.zeros(att_cnt)
    
    packets = rdpcap(pcapFilename)

    count_packet = len(packets)
    packet_open = count_packet
    
    
    if packet_open == 1:
        return 0, 0, 0, 0
    elif packet_open <= uptopacket:
        flow = np.zeros((count_packet, 1500))
        timestamp = np.zeros(count_packet)
    elif packet_open > uptopacket:
        packet_open = uptopacket
        flow = np.zeros((packet_open, 1500))
        timestamp = np.zeros(packet_open)
    else:
        print("something wrong!")
    
    packet_len = []
    
    payload = []
    
    global_att[0] = packets[0].sport
    global_att[1] = packets[0].dport
    global_att[6] = packets[count_packet-1].time - packets[0].time


    for i, data in enumerate(packets):
       
        payload.append(len(data.payload))
    
        if i < uptopacket:
    
            timestamp[i] = packets[i].time

            # mask src & dst ports to 0
            data.sport = 0
            data.dport = 0
        
            # mask src & dst IPs to 0
            if data.haslayer('Ethernet'):
                data['Ethernet'].src = '00:00:00:00:00:00'
                data['Ethernet'].dst = '00:00:00:00:00:00'
            else:
                pass
        
            if data.haslayer('IP'):
                data['IP'].src = '0.0.0.0'
                data['IP'].dst = '0.0.0.0'
            elif data.haslayer('IPv6'):
                data['IPv6'].src
                data['IPv6'].dst
                data['IPv6'].src = '0000:0000:0000:0000:0000:0000:0000:0000'
                data['IPv6'].dst = '0000:0000:0000:0000:0000:0000:0000:0000'
            else:
                pass

            
        raw_bytes = np.array(list(raw(data)))

        packet_len.append(raw_bytes.shape[0])      
        

        if raw_bytes.shape[0] >= 1500 and i<uptopacket:          
            for ii in range(1500):
                flow[i, ii] = raw_bytes[ii]

        elif raw_bytes.shape[0] < 1500 and i<uptopacket:
            blen = int(raw_bytes.shape[0])
            for ii in range(blen):
                flow[i, ii] = raw_bytes[ii]
        else:
            pass

    # mean_len = mean_len / count_packet
    global_att[4] = statistics.mean(packet_len)
    global_att[5] = statistics.stdev(packet_len)
    global_att[2] = statistics.mean(payload)
    global_att[3] = statistics.stdev(payload)
    
    return timestamp, flow, global_att